In [ ]:
import numpy as np
from matplotlib import pyplot as plt

from Alcohol.caching import load
from Processing import TimeSeries, PriceQuantity as PQ


In [ ]:
ts = load(dir_path="../Alcohol/Caches", identifier="0")
print(ts._consecutive_ranges[-1000:, 3])
longest_ts = ts.get_nth_longest(0)
top_n = [ts.get_nth_longest(i) for i in range(10)]

ts.price.shape

In [ ]:
for element in top_n:
    print("normal: ", np.corrcoef(x=element.price, y=element.quantity)[0, 1])
    print("demeaned: ", np.corrcoef(x=element.price, y=PQ.skip_demean_quantity(element, period=12).quantity)[0, 1])
    shifted = PQ.shift_price(element, 1)
    print("shifted: ", np.corrcoef(x=shifted.price, y=PQ.skip_demean_quantity(shifted, period=12).quantity)[0, 1])
    print("---------")


In [ ]:
for pq in top_n:
    pq = PQ.skip_demean_quantity(pq, 12, add_mean=True)
    ax = plt.subplots()[1]
    ax = pq.draw_scatter_graph(ax, economic_axis=True)
    #ax2 = ax.twinx() 
    ##pq.draw_scatter_graph(ax2, economic_axis=False, revenue=True, color="orange")
    # ax2.scatter(x=pq.quantity * pq.price, y=pq.price, color='orange')
    # ax2.set_xlabel("revenue [$]")

plt.show()

In [ ]:

for pq in top_n:
    #pq = PQ.skip_demean_quantity(pq, period=12)
    pq = PQ.bin_price_absolute(pq, 0.5)[:]
    ax = plt.subplots()[1]
    ax = pq.draw_std_graph(ax, economic_axis=False)
    ax2 = ax.twinx() 
    pq.draw_std_graph(ax2, economic_axis=False, revenue=True, color="orange")
 
plt.show()

In [ ]:
   
demean = True
shift = False
for i, pq in enumerate(top_n):
    if demean:
        pq = PQ.skip_demean_quantity(pq, 12, add_mean=True)
    if shift:
        pq = PQ.shift_price(pq, -1)
    print(f'{i} ', pq.get_correlation(), pq.header)
    pq.draw_time_series_scatter_graph(plt.subplots()[1], dates=True)
    plt.show()  
5
        

In [ ]:
from Processing import ModelPQ

#hezký
#index, item_number
#0, 43244
#1, 84496
def nice_res(d):
    newd = d["PED"]
    newd.update({"revenue": d["revenue"], "datetime": d["datetime"]})
    return newd

out = dict()
for i, pq in enumerate(top_n):
    pq = PQ.bin_price_absolute(pq, 0.5)[:]
    print(pq.price)
    print(pq.header)
    #print(pq.quantity)
    #print(find_change_range(pq))
    # (
    #     last_change_slope(pq, min_count=8),
    #     last_change_with_time_slope(pq, min_count=15),
    #     linear_model_slope(pq),
    #     linear_model_with_time_slope(pq)
    #  )
    
    model = ModelPQ(pq, min_count=10)
    model.run_models()
    
    out[pq.header] = tuple(nice_res(res) for res in model.results)
out